In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import time
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from threading import Thread, Lock
import threading
import pickle
# import undetected_chromedriver.v2 as uc
import datetime


In [17]:
# Сохраняем всю ленту новостей

def get_source_html(url, driver):
    counter = 0
    driver.get(url)

    time.sleep(30)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    action = ActionChains(driver)
    el = driver.find_element(By.CLASS_NAME, 'list-more')
    action.move_to_element_with_offset(el, 5, 5)
    action.click()
    action.perform()

    time.sleep(0.5)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:
        counter += 2

        while True:

            if counter >= 300 and counter % 20 == 0:
                with  open(f'ria_economy/raw_preview/news_ria{counter}.html', 'w') as file:
                    file.write(driver.page_source)

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            counter += 1

    except Exception as _ex:
        print(_ex)

    finally:
        driver.close()
        driver.quit()

In [21]:
# Сохраняем ссылки из ленты новостей

def get_file_urls(file_path):
    import os
    files = sorted(os.listdir(file_path))[1:]

    urls = []
    c = 1
    for f in files:

        with open(file_path + f'/{f}') as file:
            src = file.read()
        
        soup = BeautifulSoup(src, 'lxml')
        items_divs = soup.find_all('div', class_='list-item')

        for item in items_divs:
            item_url = item.find('div', class_='list-item__content').find('a').get('href')
            if item_url not in urls:
                urls.append(item_url)

        print(f'Ссылки из файлов {c} из {len(files)} получены')
        c+=1

    with open(f'ria_economy/urls.txt', 'w') as file:
        for url in urls:
            file.write(f'{url}\n')

        print(f'Ссылки сохранены')
    return

In [20]:
# Запускаем функцию для сбора ссылок, останавливаем вручную, потому что нельзя листать ее до бесконечности.

driver = webdriver.Chrome(ChromeDriverManager().install())

get_source_html(url='https://ria.ru/politics/', driver=driver)

/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_810/1943467916.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


KeyboardInterrupt: 

In [37]:
# Вытаскиваем ссылки
file_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_politics/raw_preview'
get_file_urls(file_path)

Ссылки из файлов 1 из 23 получены
Ссылки из файлов 2 из 23 получены
Ссылки из файлов 3 из 23 получены
Ссылки из файлов 4 из 23 получены
Ссылки из файлов 5 из 23 получены
Ссылки из файлов 6 из 23 получены
Ссылки из файлов 7 из 23 получены
Ссылки из файлов 8 из 23 получены
Ссылки из файлов 9 из 23 получены
Ссылки из файлов 10 из 23 получены
Ссылки из файлов 11 из 23 получены
Ссылки из файлов 12 из 23 получены
Ссылки из файлов 13 из 23 получены
Ссылки из файлов 14 из 23 получены
Ссылки из файлов 15 из 23 получены
Ссылки из файлов 16 из 23 получены
Ссылки из файлов 17 из 23 получены
Ссылки из файлов 18 из 23 получены
Ссылки из файлов 19 из 23 получены
Ссылки из файлов 20 из 23 получены
Ссылки из файлов 21 из 23 получены
Ссылки из файлов 22 из 23 получены
Ссылки из файлов 23 из 23 получены
Ссылки сохранены


In [2]:
# Загружаем сохраненные ссылки
with open('ria_politics/urls.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [2]:
# Создаем набор user agent, чтобы сайт не так быстро блочил
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=10000)

In [3]:
# Создаем массив данных, куда будут все склвдывать потоки
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

NameError: name 'urls' is not defined

In [4]:
# Пишем функцию для парсинга нужных данных с многопоточностью.

def parse_from_link(link, user_agent, current_value):
    if current_value % 10 == 0:
        user_agent = user_agent_rotator.get_random_user_agent()

    try:
        with requests.session() as s:
            txt = s.get(link, headers={'User-Agent':user_agent}).text
            soup = BeautifulSoup(txt, 'lxml')
            try:
                timestamp = soup.find('div', class_='article__info-date').find('a').text
            except:
                timestamp = 'No time'
            
            try:
                title = soup.find('h1', class_='article__title').text
            except:
                title = 'No title'
            try:
                announce = soup.find('div', class_='article__announce-text').text
            except:
                announce = 'No announce'
            try:
                text = soup.find('div', class_='article__body js-mediator-article mia-analytics').text
            except:
                text = 'No text'
            return timestamp, title, announce, text

    except Exception as ex:
        print(link)
        print(ex)
        return ex

    except KeyboardInterrupt:
        return


def parse(lock, links, save_evry):

    global data, total_news, current_value

    user_agent = user_agent_rotator.get_random_user_agent()

    while current_value < total_news:
        with lock:
            thread_value = current_value
            link = links[current_value]
            current_value += 1
        
        parsed = parse_from_link(link, user_agent, thread_value)
        if parsed == None:
            return

        with lock:
            data['link'].append(link)
            data['data_or_ex'].append(parsed)
            if thread_value % save_evry == 0:
                with open(f'ria_economy/raw_cites/{thread_value}_skipped.pkl', 'wb') as f:
                    pickle.dump(data, f)
                    print(f'{thread_value} saved')
                data.clear()
                data['link'] = []
                data['data_or_ex'] = []

        time.sleep(2)

    with open(f'ria_economy/raw_cites/{thread_value}_skipped.pkl', 'wb') as f:
        pickle.dump(data, f)

    return 'Конец'

def start_parsing(links, n_threads=3, save_evry=15000):
    
    lock = Lock()
    threads = []
    run_event = threading.Event()
    run_event.set()

    for i in range(n_threads):
        t = Thread(target=parse, args=(lock, links, save_evry))
        t.start()
        time.sleep(2)
        threads.append(t)

    for thread in threads:
        thread.join()

In [ ]:
# Запускаем функцию
start_parsing(urls, n_threads=2, save_evry=200)

### Дальше по сути тот же код, что и был выше, но я там докачивал пропуски

In [13]:
with open('ria_politics/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [14]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
400 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?


In [6]:
with open('ria_economy/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [7]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
400 saved
600 saved
800 saved
1000 saved
1200 saved
1400 saved
1600 saved
1800 saved
2000 saved
2200 saved
2400 saved
2600 saved
2800 saved
3000 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?


In [ ]:
with open('ria_politics/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [24]:
def urls_left(file_path):
    urls = []
    files = sorted(os.listdir(f'{file_path}'))
    
    for file in files:
        try:
            df = pd.DataFrame(pd.read_pickle(f'{file_path}/{file}'))
            df['tp'] = df['data_or_ex'].map(lambda x: isinstance(x, tuple))
            df.drop(df[df.tp == True].index, inplace=True)
            df = df[df['link'].str.contains('https://ria.ru/')==True]
            url = df['link'].astype(str).to_list()
            urls.extend(url)
        except Exception as ex:
            print(ex)
    return urls

In [21]:
politics_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_politics/raw_cites'
economics_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_economy/raw_cites'

In [25]:
urls_politics = urls_left(politics_path)
urls_economics = urls_left(economics_path)

All arrays must be of the same length
All arrays must be of the same length
invalid load key, '\x00'.
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!


In [26]:
with open(f'ria_politics/urls_retry.txt', 'w') as file:
    for url in urls_politics:
        file.write(f'{url}\n')

with open(f'ria_economy/urls_retry.txt', 'w') as file:
    for url in urls_economics:
        file.write(f'{url}\n')

In [2]:
pd.read_csv('ria_politics/prep_cites/200.csv')

,link,date,title,announce,text
0,https://ria.ru/20110211/333378123.html,19:57 11.02.2011,активист хабаровск приносить консульство япони...,активист молодежный движение хабаровск приноси...,NaN
1,https://ria.ru/20110211/333385555.html,20:13 11.02.2011,токио протестовать против медведев снова поеха...,выражать серьезный протест отношение визит сос...,москва 11 фев риа новость япония категорически...
2,https://ria.ru/20110211/333377675.html,19:56 11.02.2011,иран назло весь враг,иран отмечать 32 годовщина исламский революция...,иран отмечать 32 годовщина исламский революция...
3,https://ria.ru/20110211/333366199.html,19:26 11.02.2011,российский нация должный состоять самобытный н...,большой задача касаться формирование будущий и...,уфа 11 фев риа новость президент рф дмитрий ме...
4,https://ria.ru/20110211/333358165.html,19:05 11.02.2011,медведев называть межнациональный единство пер...,межнациональный единство мочь государство оно ...,уфа 11 фев риа новость руководитель весь регио...
...,...,...,...,...,...
194,https://ria.ru/20110203/329935025.html,17:32 03.02.2011,рф подписывать соглашение ущерб свой безопасно...,разумеется договор сокращение наступательный в...,москва 3 фев риа новость весь решение сфера бе...
195,https://ria.ru/20110203/329932778.html,17:27 03.02.2011,чиновник должный рассказывать свой работа теле...,считать представитель любой властный структура...,москва 3 фев риа новость премьер министр рф вл...
196,https://ria.ru/20110203/329922066.html,16:59 03.02.2011,путин заявлять спокойно относиться критика вра...,слушать некоторый вещь знать самый дело жизнь ...,москва 3 фев риа новость премьер министр рф вл...
197,https://ria.ru/20110203/329921841.html,16:58 03.02.2011,путин говорить дружить свой дочь,премьер признаваться удаваться дружить дочь ра...,москва 3 фев риа новость премьер министр рф вл...


In [5]:
urls = []
with open('/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_economy/raw_preview/news_ria14oct15.html') as file:
    src = file.read()

soup = BeautifulSoup(src, 'lxml')
items_divs = soup.find_all('div', class_='list-item')

for item in items_divs:
    item_url = item.find('div', class_='list-item__content').find('a').get('href')
    if item_url not in urls:
        urls.append(item_url)

In [6]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [8]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
oil_cost_november_01112015
Invalid URL 'oil_cost_november_01112015': No scheme supplied. Perhaps you meant http://oil_cost_november_01112015?
currency_november_01112015
Invalid URL 'currency_november_01112015': No scheme supplied. Perhaps you meant http://currency_november_01112015?
stock_market_november_01112015
Invalid URL 'stock_market_november_01112015': No scheme supplied. Perhaps you meant http://stock_market_november_01112015?
400 saved
600 saved
summit_GECF_23112015
Invalid URL 'summit_GECF_23112015': No scheme supplied. Perhaps you meant http://summit_GECF_23112015?
800 saved
1000 saved
Manila_APEC_summit_17112015
Invalid URL 'Manila_APEC_summit_17112015': No scheme supplied. Perhaps you meant http://Manila_APEC_summit_17112015?
1200 saved
1400 saved
1600 saved
1800 saved
2000 saved
2200 saved
2400 saved
2600 saved
2800 saved
3000 saved
oil_cost_october_01102015
Invalid URL 'oil_cost_october_01102015': No scheme supplied. Perhaps you meant http://oil_cost_oct